In [2]:
import os
folders = os.listdir("Data/Train")
print(folders)

['Bulbasaur', 'Aerodactyl', 'Dratini', 'Fearow', 'Meowth', 'Pikachu', 'Charmander', 'Squirtle', 'Spearow', 'Psyduck']


In [3]:
image_data = []
labels = []

label_dict = {
    "Bulbasaur":0,
    "Aerodactyl":1,
    "Dratini":2,
    "Fearow":3,
    "Meowth":4,
    "Pikachu":5,
    "Charmander":6,
    "Squirtle":7,
    "Spearow":8,
    "Psyduck":9
    }

In [5]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

train_file = []       
train_label = []

for dirname, _, filenames in os.walk('Data/Train'):
    for filename in filenames:
        train_file.append(os.path.join(dirname, filename))
        train_label.append(dirname.split('/')[-1])
        
train_image = pd.DataFrame(train_file)
train_image.columns = ['file_name']
train_image['target'] = train_label

print (train_image)




train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True, 
                                    validation_split = 0.2)

train_generator = train_datagen.flow_from_dataframe(
        train_image,
        x_col='file_name',
        y_col='target',
        target_size=(224, 224),
        batch_size=64,
        shuffle=True,
        class_mode='categorical',
        subset='training')

test_generator = train_datagen.flow_from_dataframe(
        train_image,
        x_col='file_name',
        y_col='target',
        target_size=(224, 224),
        batch_size=64,
        shuffle=True,
        class_mode='categorical',
        subset='validation')

2023-07-27 12:59:09.975315: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


                                              file_name     target
0                     Data/Train/Bulbasaur/00000165.jpg  Bulbasaur
1                     Data/Train/Bulbasaur/00000103.png  Bulbasaur
2     Data/Train/Bulbasaur/aa9b196e27694c9e96df3221b...  Bulbasaur
3                     Data/Train/Bulbasaur/00000192.jpg  Bulbasaur
4     Data/Train/Bulbasaur/c4e57436ca6a4a789a56eb880...  Bulbasaur
...                                                 ...        ...
1642  Data/Train/Psyduck/b2af859ecf9c49128cc63e463ed...    Psyduck
1643  Data/Train/Psyduck/b2af859ecf9c49128cc63e463ed...    Psyduck
1644  Data/Train/Psyduck/b2af859ecf9c49128cc63e463ed...    Psyduck
1645  Data/Train/Psyduck/0316cd35ef394a0897ebccde5c2...    Psyduck
1646  Data/Train/Psyduck/e21b7c660d9a47d2a15d4406207...    Psyduck

[1647 rows x 2 columns]
Found 1314 validated image filenames belonging to 10 classes.
Found 328 validated image filenames belonging to 10 classes.


/home/bhaskar/anaconda3/lib/python3.10/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 5 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  warnings.warn(
/home/bhaskar/anaconda3/lib/python3.10/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 5 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  warnings.warn(


In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import Model
from keras.layers import GlobalAveragePooling2D


#Transfer learning from ResNet50 with imagenet weights
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
X = GlobalAveragePooling2D()(model.output)
X = Dense(1000,activation = 'relu')(X)
X = Dense(500,activation='relu')(X)
output = Dense(149,activation='softmax')(X)

final_model = Model(inputs=model.input,outputs = output)

2023-07-27 13:00:30.286915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 13:00:30.464346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 13:00:30.464517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-27 13:00:30.465031: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

94781440/94765736 [==============================] - 10s 0us/step


In [7]:
#Compile the model
final_model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['acc'])


In [8]:
batch_size=16
train_steps = train_image.shape[0]//batch_size
history = final_model.fit_generator(
    train_generator,
    
    validation_data = test_generator,
    epochs=15)


/home/bhaskar/anaconda3/lib/python3.10/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15


/home/bhaskar/.local/lib/python3.10/site-packages/PIL/Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
2023-07-27 13:01:00.041081: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2023-07-27 13:01:00.848832: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 676.06MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-27 13:01:00.848873: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 676.06MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-27 13:01:01.022363: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Alloc

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2_block1_3_bn/FusedBatchNormV3' defined at (most recent call last):
    File "/home/bhaskar/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/bhaskar/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/bhaskar/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/bhaskar/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/bhaskar/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/bhaskar/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/bhaskar/anaconda3/li